In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import math
%matplotlib inline

In [3]:
df = pd.read_csv("/content/insurance_data.csv")

In [4]:
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [6]:
# splitting the dataset into training and testing dataset
X_train,X_test , y_train , y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance , test_size = 0.2 , random_state = 43)

In [9]:
X_train.shape , X_test.shape

((22, 2), (6, 2))

In [10]:
X_train.head()

,age,affordibility
18,19,0
8,62,1
27,46,1
5,56,1
24,50,1


In [22]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [23]:
X_train_scaled.head()

,age,affordibility
18,0.19,0
8,0.62,1
27,0.46,1
5,0.56,1
24,0.50,1


In [24]:
X_test_scaled.head()

,age,affordibility
12,0.27,0
22,0.40,1
10,0.18,1
6,0.55,0
7,0.60,0


# USING PACKAGES


In [29]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape = (2,), activation='sigmoid',kernel_initializer = 'ones', bias_initializer = 'zeros' )
])

In [30]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['accuracy']
)

In [32]:
model.fit(X_train_scaled, y_train,epochs = 5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 9ms/step - loss: 0.5173 - accuracy: 0.8636
Epoch 2502/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5173 - accuracy: 0.8636
Epoch 2503/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5173 - accuracy: 0.8636
Epoch 2504/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5172 - accuracy: 0.8636
Epoch 2505/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5172 - accuracy: 0.8636
Epoch 2506/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5171 - accuracy: 0.8636
Epoch 2507/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5171 - accuracy: 0.8636
Epoch 2508/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5170 - accuracy: 0.8636
Epoch 2509/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5170 - accuracy: 0.8636
Epoch 2510/5000
1/1 [==============================]

In [33]:
model.evaluate(X_test_scaled , y_test)

1/1 [==============================] - 2s 2s/step - loss: 0.4368 - accuracy: 1.0000


[0.43678930401802063, 1.0]

In [34]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 271ms/step


array([[0.17831561],
       [0.6025649 ],
       [0.32495296],
       [0.48309517],
       [0.5481249 ],
       [0.76823825]], dtype=float32)

In [35]:
coef , intercept = model.get_weights()

In [37]:
print(coef[0], coef[1])
print(intercept )

[5.2148438] [1.2660353]
[-2.9358091]


# FROM SCATCH

In [39]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [44]:
def prediction_function( age, aff):
  y = age*coef[0] + aff*coef[1] + intercept
  yhat = sigmoid(y)
  return yhat

# testing
prediction_function(0.27,0)

0.17831561325140374

In [62]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [63]:
def sigmoid_numpy(x):
  return 1/(1+np.exp(-x))

In [66]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break

    return w1, w2, bias

In [68]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train ,1000,0.4375)

Epoch:0, w1:0.9755342677219864, w2:0.9343049617339481, bias:-0.1210471021515617, loss:0.7229866450171974
Epoch:1, w1:0.9572952952908756, w2:0.880453130983193, bias:-0.2259503534026453, loss:0.6866919324412373
Epoch:2, w1:0.9447997509740013, w2:0.8376482359739109, bias:-0.3160704190939209, loss:0.6604409523673477
Epoch:3, w1:0.9374048253268601, w2:0.8047199386951234, bias:-0.3931254780683189, loss:0.6418704234624636
Epoch:4, w1:0.9344032729224915, w2:0.7803257118964337, bias:-0.458956781420467, loss:0.6288886056007145
Epoch:5, w1:0.9350973957293062, w2:0.7631120101695599, bias:-0.5153519355765014, loss:0.6198168532693934
Epoch:6, w1:0.9388454022979359, w2:0.7518179662843999, bias:-0.5639380298961366, loss:0.6134009914676587
Epoch:7, w1:0.9450838869888639, w2:0.745328171974969, bias:-0.6061330242022206, loss:0.6087512890806388
Epoch:8, w1:0.953333757008475, w2:0.7426895998588574, bias:-0.6431364587084469, loss:0.6052595753351859
Epoch:9, w1:0.9631964550810905, w2:0.7431071253787629, bias

(5.171805511312905, 1.407718460366434, -3.0525722560009347)

In [69]:
coef , intercept

(array([[5.2148438],
        [1.2660353]], dtype=float32),
 array([-2.9358091], dtype=float32))

# NEURAL NETWORK USING SCRATCH

In [93]:
class myNN():
  def __init__(self):
    self.w1 = 1
    self.w2 = 1
    self.bias = 0

  def fit(self , X , y , epochs , loss_threshold):
    self.w1, self.w2 , self.bias = self.gradient_descent(X['age'], X['affordibility'], y , epochs , loss_threshold)

  def predict(self, X_test):
    weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
    return sigmoid_numpy(weighted_sum)



  def gradient_descent(self,age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        if i%50==0:
          print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
            break

    return w1, w2, bias




In [94]:
model2 = myNN()

In [95]:
model2.fit(X_train_scaled, y_train,epochs = 500, loss_threshold = 0.4375)

Epoch:0, w1:0.9755342677219864, w2:0.9343049617339481, bias:-0.1210471021515617, loss:0.7229866450171974
Epoch:50, w1:1.5878595611748771, w2:1.078828499079195, bias:-1.3183594861140797, loss:0.555109734225093
Epoch:100, w1:2.3357158354181538, w2:1.2996321882004958, bias:-1.8103951804373446, loss:0.520678635166239
Epoch:150, w1:3.029871573819802, w2:1.3930077974497816, bias:-2.1729746061200017, loss:0.4956358985849975
Epoch:200, w1:3.6699646989210875, w2:1.4261674303859786, bias:-2.461414301433154, loss:0.4757865318068197
Epoch:250, w1:4.258619276222539, w2:1.4299763859666712, bias:-2.7030634055150937, loss:0.4595289104518996
Epoch:300, w1:4.799651102430389, w2:1.419671364872306, bias:-2.9127567584918914, loss:0.44601123088188815
Epoch:337, w1:5.171805511312905, w2:1.407718460366434, bias:-3.0525722560009347, loss:0.4374352124941607


In [96]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 22ms/step


array([[0.17831561],
       [0.6025649 ],
       [0.32495296],
       [0.48309517],
       [0.5481249 ],
       [0.76823825]], dtype=float32)